## Step 2: Model Training

In this notebook, we'll build and train a Convolutional Neural Network (CNN) to classify images as real or fake. We will use PyTorch for this task.

### 2.1: Install and Import Libraries

We need `torch` for building the model and `torchvision` for image transformations. We'll also need `datasets` again to load our data.

In [ ]:
%pip install datasets torch torchvision

In [ ]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

### 2.2: Pre-process the Data

Neural networks require images of a consistent size. We'll resize all images to 128x128 pixels. We also need to convert them to PyTorch tensors and normalize their pixel values.

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize to range [-1, 1]
])

def apply_transforms(examples):
    examples['pixel_values'] = [data_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

### 2.3: Load and Transform the Dataset

We'll load the dataset again and apply our transformations. To speed up training for this example, we'll only use a subset of the data.

In [ ]:
# Load the dataset
dataset = load_dataset("saakshigupta/deepfake-detection-dataset-v3")

# Apply the transformations
transformed_dataset = dataset.with_transform(apply_transforms)

# Create DataLoaders
train_loader = DataLoader(transformed_dataset['train'], batch_size=32, shuffle=True)
test_loader = DataLoader(transformed_dataset['test'], batch_size=32, shuffle=False)

### 2.4: Define the CNN Model

Here is a simple CNN architecture. It consists of several convolutional layers to extract features from the images, followed by fully connected layers to make the final classification.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 16 * 16, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid() # Sigmoid for binary classification (0 or 1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) # Flatten the tensor
        x = self.fc_layers(x)
        return x

### 2.5: Train the Model

Now, let's set up the training loop. We'll define a loss function (Binary Cross-Entropy for this classification task) and an optimizer (Adam is a good default). We'll train for a few epochs and print the loss at each step.

In [ ]:
model = SimpleCNN()
criterion = nn.BCELoss() # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3 # We'll train for 3 epochs as a start

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs = batch['pixel_values']
        labels = batch['label'].float().view(-1, 1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0: # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training!')

### 2.6: Save the Model

After training, it's a good practice to save the model's weights. This allows us to use it later for evaluation or inference without having to retrain it.

In [ ]:
torch.save(model.state_dict(), 'deepfake_detector_model.pth')
print("Model saved to deepfake_detector_model.pth")

## Next Steps

Now that we have trained a model, the next step is to evaluate its performance on the test set. We'll create a new notebook, `3_Model_Evaluation.ipynb`, to see how well our model can distinguish real from fake on unseen data.